# <div style="text-align: center; background-color:lightgreen; font-family:Georgia, serif; color: black; padding: 20px;line-height: 1;border-radius:5px; border: 2px solid black;">Hotel Bookings Analysis</div>

#### This project presents a comprehensive analysis of hotel booking data to uncover key booking patterns, optimize revenue strategies, and reduce cancellation rates. By leveraging data-driven insights, the project aims to enhance operational efficiency, improve the guest experience, and maximize overall profitability. The analysis focuses on identifying factors influencing booking behaviour, evaluating the performance of different distribution channels, and providing actionable recommendations to support data-driven decision-making.

## Project Objectives

The primary objectives of this project include:

*	Identifying factors contributing to booking cancellations, enabling the hotel to proactively reduce cancellation rates.

*	Analyzing revenue patterns across customer segments, providing insights into the most valuable customer profiles.
	
*	Optimizing pricing strategies by analyzing the relationship between lead times, seasonal demand, and average daily rates.
	
*	Evaluating the effectiveness of different distribution channels to determine the most profitable booking sources.


## Importing Libraries

In [93]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [94]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df=pd.read_excel('Row Data.xlsx')

In [ ]:
df

## Data Overview
The analysis is based on a comprehensive hotel booking dataset, which includes the following key fields:

* Booking ID: Unique identifier assigned to each booking
* Hotel: Property type where the booking was made
* Booking Date: Date when the reservation was created
* Arrival Date: Scheduled check-in date
* Lead Time: Number of days between booking and arrival date
* Nights: Duration of the stay in nights
* Guests: Number of guests included in the booking
* Distribution Channel: Source of booking (Direct, Travel Agent, Corporate, etc.)
* Customer Type: Classification of customers (Transient, Group, etc.)
* Country: Country of origin of the guest
* Deposit Type: Deposit payment status (No Deposit, Non-Refundable, etc.)
* Average Daily Rate (ADR): Average price per night charged for the booking
* Status: Current status of the booking (Check-Out, Cancelled, etc.)
* Status Update: Date when the booking status was last modified
* Cancelled (0/1): Indicator denoting whether the booking was cancelled
* Revenue: Actual revenue generated from completed bookings
* Revenue Loss: Estimated revenue loss due to cancellations


<left><h1 style="color:darkred; font-size: 36px;">Exploratory Data Analysis (EDA)</h1></left>

In [ ]:
df.size

In [ ]:
df.shape

## Informations of the dataset
* Total Columns= 17
* Total Rows= 119390
* Data Types
    - Integer= 5
    - Float= 3
    - Datetime= 3
    - Object= 6


In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.describe(include='object')

In [ ]:
df.nunique()

In [ ]:
df.ndim

### Checking Duplicates

In [ ]:
df.duplicated().sum()

> #### No duplicated rows

### Checking Null Values

In [ ]:
df.isnull().sum()

> #### Country column has 488 null values

#### Filling null values using mode()

In [ ]:
df['Country']=df['Country'].fillna(df['Country'].mode()[0])

In [ ]:
df.isna().sum()

# Feature Engineering

## 1. Time-Based Columns:

### Extracted Year of Booking,Month of Booking, Dayname of Booking from Booking Date

In [ ]:
df['Year of Booking']=df['Booking Date'].dt.year

In [ ]:
df['Month of Booking']=df['Booking Date'].dt.month_name()

In [ ]:
df['Weekday of Booking']=df['Booking Date'].dt.day_name()

### Extracted year,Month name, Dayname from Arrival Date

In [ ]:
df['Year of Arrival']=df['Arrival Date'].dt.year

In [ ]:
df['Month of Arrival']=df['Arrival Date'].dt.month_name()

In [ ]:
df['Weekday of Arrival']=df['Arrival Date'].dt.day_name()

## 2. Booking Behavior Columns:

### Booking Window Category

In [ ]:
print(df[df['Lead Time'].isna()])

In [ ]:
df['Lead Time'].unique()

In [ ]:
df['Lead Time'].max()

In [ ]:
df['Lead Time'].min()

In [ ]:
df['Booking Window Category']=pd.cut(df['Lead Time'],bins=[0,7,30,180,737],labels=['Last Minute','Short Term','Long Term','Very Long Term'],include_lowest=True)

In [ ]:
df['Booking Window Category'].value_counts(dropna=False)

### Stay Duration Category

In [ ]:
df['Nights'].unique()

In [ ]:
df['Nights'].max()

In [ ]:
df['Stay Duration Category']=pd.cut(df['Nights'],bins=(0,3,8,30,69),labels=['Short Stay','Medium Stay','Long Stay','Very Long Stay'],include_lowest=True)

In [ ]:
df['Stay Duration Category'].value_counts()

### Guest Type

In [ ]:
df['Guests'].unique()

In [ ]:
df['Guests'].min()

In [ ]:
df['Guests'].max()

In [ ]:
df['Guests'].value_counts()

In [ ]:
def category_guest(guest):
    if guest==0:
        return 'Invalid Entry'
    elif guest==1:
        return 'Solo'
    elif guest==2:
        return 'Couple'
    elif guest>3 and guest<6:
        return 'Family'
    else:
        return 'Groups'

df['Guest Type']=df['Guests'].apply(category_guest)

In [ ]:
df['Guest Type'].value_counts()

## 3. Customer Segmentation Columns:

### Repeat Guest

In [ ]:
df.duplicated(subset=['Booking ID']).value_counts()

> #### There is no Repeat Guests

### High-Value Customers

In [ ]:
df['Total Booking Value'] = df['Avg Daily Rate'] * df['Nights'] * df['Guests']

In [ ]:
# Compute the 90th percentile of Total Booking Value
threshold = df['Total Booking Value'].quantile(0.90)

# Assign High-Value Customer flag dynamically
df['High-Value Customer'] = (df['Total Booking Value'] > threshold).astype(int)


In [ ]:
df['High-Value Customer'].value_counts()

### Profit

In [ ]:
df['Profit']=df['Revenue']-df['Revenue Loss']

## Save the modified data to a csv file

In [ ]:
df.to_csv('Hotel Booking Data.csv',index=False)

<center><h1 style="color:darkblue; font-size: 36px;">Hotel Booking Analysis</h1></center>



In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df=pd.read_csv('Hotel Booking Data.csv')

In [ ]:
df

<left><h1 style="color:darkred; font-size: 36px;">Univariate Analysis</h1></left>

<left><h5 style="color:green; font-size: 36px;">Boxplot</h5></left>

In [ ]:
plt.figure(figsize=(15,8))
df.boxplot()
plt.xticks(rotation=45)
plt.show()

> #### Booking ID has a wide range — it's just an identifier, not useful for analysis.

#### Lead Time, ADR, Revenue, Total Booking Value, and Profit have:

* High values for a few bookings (outliers).

* Most bookings have lower values.

#### Revenue Loss shows some negative values — might be errors.

#### Cancelled, Repeat Guest, and High-Value Customer are mostly 0 or 1 — they are yes/no type data.

<left><h5 style="color:green; font-size: 36px;">Heatmap</h5></left>

In [ ]:
numer=df.select_dtypes(['int','float'])
plt.figure(figsize=(15,8))
sns.heatmap(numer.corr(),annot=True)
plt.show()

> #### Profit is most strongly correlated with Total Booking Value (0.91), followed by Nights (0.75) and Revenue (0.63), while cancellations negatively affect both revenue and profit.

<left><h5 style="color:green; font-size: 36px;">Analysis of Accommodation Type</h5></left>

In [ ]:
df['Hotel'].value_counts()

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(x=df['Hotel'],color='purple')
plt.title('Comparison of Accommodation Type',fontsize=14, fontweight='bold')
plt.xlabel("Accommodation Type")
plt.ylabel("Number of Bookings")
plt.savefig("accommodation type.png", dpi=300, bbox_inches='tight')
plt.show()

> #### The number of hotel bookings (79,330) is significantly higher than resort bookings (40,060)

<left><h5 style="color:green; font-size: 36px;">Distribution Channel Analysis</h5></left>

In [ ]:
df['Distribution Channel'].value_counts()

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(x='Distribution Channel',data=df,color='purple')
plt.title('Distribution Channel Analysis: Booking Preferences',fontsize=14, fontweight='bold')
plt.savefig("distribution channel.png", dpi=300, bbox_inches='tight')
plt.show()

> #### The majority of bookings come from online travel agents (74,072), indicating a strong preference for digital platforms, while corporate and undefined channels contribute the least.

<left><h5 style="color:green; font-size: 36px;">Customer Type Analysis</h5></left>

In [ ]:
df['Customer Type'].value_counts()

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(x='Customer Type',data=df,color='purple')
plt.title('Customer Type Analysis: Dominance of Transient Bookings',fontsize=14, fontweight='bold')
plt.savefig("customer type.png", dpi=300, bbox_inches='tight')
plt.show()

> #### Transient customers (89,613) form the largest segment, highlighting a preference for short-term, individual stays, while group bookings (577) are the least common. 

<left><h5 style="color:green; font-size: 36px;">Analysis of Countries by Hotel Booking</h5></left>

In [ ]:
df['Country'].value_counts()

In [ ]:
plt.figure(figsize=(10,5))
plt.title('Top 10 Countries by Hotel Bookings',fontsize=14, fontweight='bold')
df['Country'].value_counts().head(10).plot(kind='bar',color='purple')
plt.savefig("top country by booking.png", dpi=300, bbox_inches='tight')
plt.show()

> #### Portugal (49,078) leads with the highest number of bookings, likely due to domestic travelers, while the United Kingdom (12,129) and France (10,415) follow as key international markets, reflecting strong European tourism demand

## Least Represented Countries

In [ ]:
df['Country'].value_counts().tail(10)

> #### Countries like Madagascar, Myanmar, and Nepal each have only one booking, indicating minimal travel activity from these regions, possibly due to geographical distance, visa restrictions, or lower outbound tourism rates.

<left><h5 style="color:green; font-size: 36px;">Analysis of Deposit Payment Status</h5></left>

In [ ]:
df['Deposit Type'].value_counts()

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(x='Deposit Type',data=df,color='purple')
plt.title('Deposit Type Preference in Hotel Bookings',fontsize=14, fontweight='bold')
plt.show()

> #### The vast majority of bookings (104,641) are made with no deposit, suggesting a preference for flexible payment options, while non-refundable (14,587) and refundable (162) bookings are significantly lower.

<left><h5 style="color:green; font-size: 36px;">Guest Stay Status Distribution</h5></left>

In [ ]:
df['Status'].value_counts()

In [ ]:
df['Status'].value_counts().plot(kind='pie',autopct="%1.2f%%",explode=(0,0,0.1),colors=['purple', '#9B59B6', '#B57EDC'])
plt.title('Guest Stay Status Distribution',fontsize=14, fontweight='bold')
plt.show()

> #### Most guests completed their stay (63% Check-Out), while a significant portion (36% Canceled) canceled their bookings, and only 1% were No-Shows, indicating high booking commitment overall.

<left><h5 style="color:green; font-size: 36px;">Status Update Frequency Over Time</h5></left>

In [ ]:
df['Status Update'].value_counts()

> #### A few specific dates, like 2015-10-21, show spikes in status updates, indicating bulk actions or peak activity periods, while most dates had minimal changes, suggesting scattered individual updates.

<left><h5 style="color:green; font-size: 36px;">Booking Cancellation Distribution</h5></left>

In [ ]:
df['Cancelled (0/1)'].value_counts()

In [ ]:
df['Cancelled (0/1)'].value_counts().plot(kind='pie',autopct="%1.2f%%",colors=['purple', '#9B59B6'],labels=['Not Cancelled', 'Cancelled'],labeldistance=0.3)

plt.title('Booking Cancellation Distribution',fontsize=14, fontweight='bold')
plt.show()

> #### Around 37% of bookings were cancelled, while 63% were completed, showing a moderately high cancellation rate that may warrant further analysis of influencing factors.

<left><h5 style="color:green; font-size: 36px;">Guest Booking Window Preferences</h5></left>

In [ ]:
df['Booking Window Category'].value_counts()

In [ ]:
df['Booking Window Category'].value_counts().plot(kind='pie',autopct="%1.2f%%",colors=['purple', '#9B59B6', '#B57EDC','#D8BFD8'],labeldistance=0.8)

plt.title('Guest Booking Window Preferences', fontsize=14,fontweight='bold')
plt.savefig("booking window.png", dpi=300, bbox_inches='tight')
plt.show()

> #### A majority of guests booked well in advance, with Long Term and Very Long Term bookings making up over 75% of total bookings, while Last Minute and Short Term bookings were significantly lower, indicating strong planning behavior among guests.

<left><h5 style="color:green; font-size: 36px;">Distribution of Stay Duration</h5></left>

In [ ]:
df['Stay Duration Category'].value_counts()

In [ ]:
plt.figure(figsize=(10,5))
df['Stay Duration Category'].value_counts().plot(kind='bar',color='purple')

plt.title('Distribution of Stay Duration', fontsize=14,fontweight='bold')
plt.savefig("stay duration.png", dpi=300, bbox_inches='tight')
plt.show()

> #### Short stays dominate the booking patterns, making up the vast majority of visits (over 60k), while Very Long Stays are extremely rare, indicating that most guests prefer brief stays over extended visits.

<left><h5 style="color:green; font-size: 36px;">Guest Type Distribution</h5></left>

In [ ]:
df['Guest Type'].value_counts()

In [ ]:
plt.figure(figsize=(10,5))
df['Guest Type'].value_counts().plot(kind='bar',color='purple')
plt.title('Guest Type Distribution', fontsize=14,fontweight='bold')
plt.savefig("guest type.png", dpi=300, bbox_inches='tight')
plt.show()

> #### The majority of bookings were made by couples (over 80k), followed by solo travelers and groups, while families and invalid entries made up a very small portion. This suggests the hotel is most popular among couples, potentially indicating a leisure or romantic travel preference.

<left><h5 style="color:green; font-size: 36px;">Repeat Guest Analysis</h5></left>

In [ ]:
df['Repeat Guest'].value_counts()

> #### 100% of the guests were first-time visitors, highlighting a lack of repeat business. This could indicate potential for growth in customer loyalty or a focus on one-time leisure stays.

<left><h1 style="color:darkred; font-size: 36px;">Bivariate & Comparative Analysis</h1></left>

<left><h5 style="color:green; font-size: 36px;">Booking Volume by Year</h5></left>

In [ ]:
df['Year of Booking'].value_counts()

In [ ]:
plt.figure(figsize=(10,5))
df['Year of Booking'].value_counts().plot(kind='bar',color='purple')
plt.title('Yearly Booking Trend', fontsize=14, fontweight='bold')
plt.xlabel('Year')
plt.ylabel('Number of Bookings')
plt.savefig("booking year.png", dpi=300, bbox_inches='tight')
plt.show()

> #### Most bookings were made between 2015 and 2017, with a peak in 2016 (58.5k bookings), showing strong growth compared to earlier years.

<left><h5 style="color:green; font-size: 36px;">Monthly Booking Trends</h5></left>

In [ ]:
df['Month of Booking'].value_counts()

In [ ]:
plt.figure(figsize=(10, 6))
df['Month of Booking'].value_counts().plot(kind='bar', color='purple')
plt.title('Monthly Booking Trends', fontsize=14,fontweight='bold')
plt.xlabel('Month')
plt.ylabel('Number of Bookings')
plt.savefig("booking month.png", dpi=300, bbox_inches='tight')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

> #### January and February saw the highest bookings, suggesting a seasonal surge at the beginning of the year, while June and May had the lowest, hinting at a mid-year dip in customer activity.

<left><h5 style="color:green; font-size: 36px;">Booking Trends by Day of the Week</h5></left>

In [ ]:
df['Weekday of Booking'].value_counts()

In [ ]:
plt.figure(figsize=(10,6))
df['Weekday of Booking'].value_counts().plot(kind='bar',color='purple')
plt.title('Booking Trends by Day of the Week', fontsize=14,fontweight='bold')
plt.xlabel('Day of Week')
plt.ylabel('Number of Bookings')
plt.tight_layout()
plt.savefig("booking day.png", dpi=300, bbox_inches='tight')
plt.show()

> #### Most bookings occurred on weekdays, with Friday leading the volume, while weekends (Saturday & Sunday) had the lowest activity, suggesting that customers prefer to book during the workweek.

<left><h5 style="color:green; font-size: 36px;">Guest Arrivals by Year</h5></left>

In [ ]:
df['Year of Arrival'].value_counts()

In [ ]:
plt.figure(figsize=(10,6))
df['Year of Arrival'].value_counts().plot(kind='bar',color='purple')
plt.title('Guest Arrivals by Year', fontsize=14,fontweight='bold')
plt.xlabel('Year')
plt.ylabel('Number of Arrivals')
plt.savefig("arrival year.png", dpi=300, bbox_inches='tight')
plt.tight_layout()
plt.show()

> #### 2016 had the highest number of arrivals (56.7k), followed by a drop in 2017 and significantly fewer in 2015, suggesting a growth in business leading up to 2016, with a slight decline afterward.

<left><h5 style="color:green; font-size: 36px;">Monthly Guest Arrivals</h5></left>

In [ ]:
df['Month of Arrival'].value_counts()

In [ ]:
plt.figure(figsize=(10, 6))
df['Month of Arrival'].value_counts().plot(kind='bar', color='purple')
plt.title('Monthly Guest Arrivals Trends', fontsize=14,fontweight='bold')
plt.xlabel('Month')
plt.ylabel('Number of Bookings')
plt.xticks(rotation=90)
plt.tight_layout()
plt.savefig("arrival month.png", dpi=300, bbox_inches='tight')
plt.show()

> #### August, July, and May were the peak months for guest arrivals, highlighting strong summer seasonality, while January and December had the fewest, indicating low winter travel activity.

<left><h5 style="color:green; font-size: 36px;">Guest Arrivals by Day of the Week</h5></left>

In [ ]:
df['Weekday of Arrival'].value_counts()

In [ ]:
plt.figure(figsize=(10,6))
df['Weekday of Arrival'].value_counts().plot(kind='bar',color='purple')
plt.title('Guest Arrivals by Day of the Week', fontsize=14,fontweight='bold')
plt.xlabel('Day of Week')
plt.ylabel('Number of Bookings')
plt.tight_layout()
plt.savefig("arrival day.png", dpi=300, bbox_inches='tight')
plt.show()

> #### Fridays and Thursdays had the highest arrival rates, indicating a preference for weekend or end-of-week check-ins, while Tuesday and Sunday saw the fewest, showing mid-week and post-weekend dips.

<left><h5 style="color:green; font-size: 36px;">Cancellations by Category</h5></left>

In [ ]:
plt.figure(figsize=(10,6))
sns.countplot(data=df, x='Deposit Type', hue='Cancelled (0/1)',palette=['purple', '#9B59B6'])
plt.title('Cancellation by Deposit Type', fontsize=14,fontweight='bold')
plt.tight_layout()
plt.show()

> #### Cancellations are highest for bookings with no deposit, while refundable deposits see the fewest cancellations.

In [ ]:
plt.figure(figsize=(10,6))
sns.countplot(data=df,x='Customer Type',hue='Cancelled (0/1)',palette=['purple', '#9B59B6'])
plt.title('Cancellation by Customer Type', fontsize=14,fontweight='bold')
plt.tight_layout()
plt.show()

> #### transient customers account for the majority of cancellations, while contract and group customers show minimal cancellation activity.

In [ ]:
plt.figure(figsize=(10,6))
sns.countplot(data=df, x='Distribution Channel', hue='Cancelled (0/1)',palette=['purple', '#9B59B6'])
plt.title('Cancellation by Distribution Channel', fontsize=14,fontweight='bold')
plt.tight_layout()
plt.show()

> #### cancellations are predominantly associated with the 'Online Travel Agent' distribution channel, highlighting its significant share compared to other channels.

In [ ]:
# Create a binary column for cancellation
df['Is Canceled'] = df['Status'].apply(lambda x: 1 if x == 'Canceled' else 0)

# Optionally preview cancellation rates
df.groupby(['Deposit Type', 'Customer Type', 'Distribution Channel'])['Is Canceled'].mean().reset_index()

In [ ]:
# Group and calculate cancellation rate
cancel_rate = df.groupby(['Deposit Type', 'Customer Type', 'Distribution Channel'])['Is Canceled'].mean().reset_index()

# Plot using FacetGrid
plt.figure(figsize=(10,6))
g = sns.catplot(
    data=cancel_rate,
    kind='bar',
    x='Deposit Type',
    y='Is Canceled',
    hue='Distribution Channel',
    col='Customer Type',
    palette="flare",
    height=5,
    aspect=1
)

g.set_titles("Customer Type: {col_name}")
g.set_axis_labels("Deposit Type", "Cancellation Rate")
g.set(ylim=(0, 1))
g.tight_layout()
plt.subplots_adjust(top=0.85)
g.fig.suptitle("Cancellation Rate by Deposit Type, Customer Type, and Distribution Channel",fontsize=14,fontweight='bold')
# Save the figure
g.savefig("cancellation_rate_plot.png", dpi=300, bbox_inches='tight')
plt.show()

> #### Deposit Type: 'No Deposit' leads to the highest cancellation rates, whereas bookings with refundable deposits experience the least cancellations.
> #### Customer Type: 'Transient' customers dominate cancellations, unlike 'Contract' and 'Group' customers, who display minimal activity.
> #### Distribution Channel: The 'Online Travel Agent' channel shows a notable trend of cancellations in comparison to others.


<left><h5 style="color:green; font-size: 36px;">Stay Duration vs Guest Type</h5></left>

In [ ]:
plt.figure(figsize=(10,6))
sns.countplot(data=df,x='Stay Duration Category', hue='Guest Type',palette=['purple', '#9B59B6','#B57EDC','#D8BFD8','grey'])
plt.title('Stay duration vs Guest Type', fontsize=14,fontweight='bold')
plt.tight_layout()
plt.savefig("stay duration vs guest type.png", dpi=300, bbox_inches='tight')
plt.show()

> #### Short stays are most common, especially among couples, followed by solo guests. Longer stays have fewer guests across all types.


<left><h5 style="color:green; font-size: 36px;">Booking Window vs Stay Duration</h5></left>

In [ ]:
plt.figure(figsize=(10,6))
sns.countplot(data=df,x='Booking Window Category', hue='Stay Duration Category',palette=['purple', '#9B59B6','#B57EDC','#D8BFD8'])
plt.title('Stay duration vs Booking Window', fontsize=14,fontweight='bold')
plt.tight_layout()
plt.savefig("stay duration vs booking window.png", dpi=300, bbox_inches='tight')
plt.show()

> #### Short stays dominate all booking windows, with a steady decline in longer stays across categories.


<left><h5 style="color:green; font-size: 36px;">Yearly Booking Trend</h5></left>

In [ ]:

plt.figure(figsize=(10,6))
sns.lineplot(x='Year of Booking',y='Revenue',data=df,color='purple')
plt.title('Yearly Booking Revenue', fontsize=14,fontweight='bold')

plt.savefig("Yearly Booking revenue.png", dpi=300, bbox_inches='tight')
plt.show()

> #### Yearly booking revenue shows steady growth from 2013 to 2017, with notable increases in 2014 and gradual rises afterward.

In [ ]:


plt.figure(figsize=(10,6))
sns.lineplot(x='Year of Booking',y='Profit',data=df,color='purple')
plt.title('Yearly Booking Profit', fontsize=14,fontweight='bold')

plt.savefig("Yearly Booking profit.png", dpi=300, bbox_inches='tight')
plt.show()

> #### Profits from yearly bookings show consistent growth, starting modestly in 2013 and soaring to significant highs by 2017.



<left><h5 style="color:green; font-size: 36px;">Revenue Distribution</h5></left>

In [ ]:
sns.barplot(x="Customer Type",y="Revenue",data=df,estimator='sum',color='purple')
plt.title("Revenue by Customer Type",fontsize=14,fontweight='bold')
plt.show()

> #### Transient customers generate the most revenue, while contract and group customers contribute the least. Transient-party customers fall in between.

In [ ]:
plt.figure(figsize=(10,5))
sns.barplot(x="Distribution Channel",y="Revenue",data=df,estimator='sum',color='purple')
plt.title("Revenue by Distribution Channel",fontsize=14,fontweight='bold')
plt.show()

> #### Online Travel Agent channels dominate revenue generation, significantly surpassing Offline Travel Agents, Direct, Corporate, and Undefined channels.


### Revenue by Distribution Channel and Customer Type

In [ ]:
grouped = df.groupby(['Distribution Channel', 'Customer Type'])['Revenue'].sum().reset_index()

plt.figure(figsize=(10,6))
sns.barplot(data=grouped, x='Distribution Channel', y='Revenue', hue='Customer Type', palette='flare')
plt.title("Revenue by Distribution Channel and Customer Type")
plt.ylabel("Revenue")
plt.savefig("revenue by distribution channel and customer type.png", dpi=300, bbox_inches='tight')
plt.tight_layout()
plt.show()

> #### The "Online Travel Agent" channel drives the highest revenue, especially from transient customers.

### Revenue Share by Country

In [ ]:
top=df.groupby('Country')["Revenue"].sum().sort_values(ascending=False).head(10)
top

In [ ]:
plt.figure(figsize=(10,6))
top.plot(kind='barh', color='purple')
plt.xlabel("Total Revenue")
plt.ylabel("Country")
plt.title("Top 10 Countries by Revenue",fontsize=14,fontweight="bold")
plt.gca().invert_yaxis()  # To show highest revenue on top
plt.tight_layout()
plt.savefig("top country revenue.png", dpi=300, bbox_inches='tight')
plt.show()

> #### Portugal generates the highest revenue among the top 10 countries, followed by the United Kingdom and France, with Switzerland at the lowest position.


<left><h5 style="color:green; font-size: 36px;">Average Daily Rate over Booking Months</h5></left>

In [ ]:
# Calculate average ADR by month
adr_by_month = df.groupby('Month of Booking')['Avg Daily Rate'].mean().reindex([
    'January', 'February', 'March', 'April', 'May', 'June',
    'July', 'August', 'September', 'October', 'November', 'December'
])

In [ ]:
plt.figure(figsize=(10,6))
adr_by_month.plot(kind='line', marker='o', color='purple')
plt.title("Average Daily Rate Over Booking Months")
plt.xlabel("Month")
plt.ylabel("Average Daily Rate")
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.savefig("avg dailyrevenue booking.png", dpi=300, bbox_inches='tight')
plt.show()

> #### The graph illustrates a seasonal pattern in the average daily rate, peaking in July and gradually declining through to November, with slight rebounds toward the end of the year.



<left><h5 style="color:green; font-size: 36px;">Lead Time vs. ADR</h5></left>

In [ ]:
plt.figure(figsize=(10,6))
sns.regplot(data=df, x='Lead Time', y='Avg Daily Rate', scatter=True,
            lowess=True, line_kws={'color': 'red'})
plt.title("Lead Time vs. ADR with Trend")
plt.xlabel("Lead Time (days)")
plt.ylabel("Average Daily Rate")
plt.grid(True)
plt.tight_layout()
plt.show()

> #### The scatter plot shows no significant correlation between lead time and average daily rate, as the trend remains almost flat across all data points.



<left><h5 style="color:green; font-size: 36px;">Profit by Booking Window Category</h5></left>

In [ ]:
bucketed_revenue = df.groupby('Booking Window Category')['Profit'].sum().reset_index()

In [ ]:
plt.figure(figsize=(10,6))
sns.barplot(data=bucketed_revenue, x='Booking Window Category', y='Profit', palette=['purple', '#9B59B6','#B57EDC','#D8BFD8'])
plt.title("Profit by Lead Time Buckets")
plt.xlabel("Lead Time Range")
plt.ylabel("Total Profit")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig("profit by booking window.png", dpi=300, bbox_inches='tight')
plt.show()

> #### Long-term lead time bookings generate the highest profits, followed by very long-term bookings, with last-minute bookings yielding the least.

<left><h5 style="color:green; font-size: 36px;">Revenue Gain vs. Revenue Loss</h5></left>

In [ ]:
channel_data = df.groupby('Distribution Channel')[['Revenue', 'Revenue Loss']].sum().reset_index()

In [ ]:
# Melt for grouped bar format
melted = channel_data.melt(id_vars='Distribution Channel', 
                           value_vars=['Revenue', 'Revenue Loss'],
                           var_name='Metric', value_name='Amount')

plt.figure(figsize=(10,6))
sns.barplot(data=melted, x='Distribution Channel', y='Amount', hue='Metric', palette=['purple', '#9B59B6'])
plt.title("Revenue vs. Revenue Loss")
plt.ylabel("Amount")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig("revenue vs revenue loss.png", dpi=300, bbox_inches='tight')
plt.show()

> #### Revenue is highest for the "Online Travel Agent" channel but is accompanied by significant revenue loss, while other channels have lower revenue and losses.


<left><h5 style="color:green; font-size: 36px;">Customer Flow from Distribution Channel to Status</h5></left>

In [ ]:
# Aggregate data for stacked bar
flow_data = df.groupby(['Distribution Channel', 'Status']).size().unstack().fillna(0)

# Plot stacked bar chart
color_p=['#D8BFD8', '#9B59B6','purple']
flow_data.plot(kind='bar', stacked=True, figsize=(10,6), color=color_p)

plt.title("Customer Flow from Distribution Channel to Status")
plt.xlabel("Distribution Channel")
plt.ylabel("Number of Customers")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig("Customer Flow from Distribution Channel to Status.png", dpi=300, bbox_inches='tight')
plt.show()

> #### The chart shows that the "Online Travel Agent" channel has the highest customer flow, with most customers successfully checking out, while no-shows and cancellations are less frequent.

<left><h1 style="color:darkred; font-size: 36px;">Findings</h1></left>



1. **Booking Preferences:** Hotel bookings significantly outnumber resort bookings, and transient customers form the largest segment, with couples leading among guest types.
2. **Booking Channels:** Online travel agents dominate bookings and revenue generation, but also experience significant cancellations and revenue losses.
3. **Guest Behavior:** Most guests book well in advance, prefer short stays, and choose flexible payment options like no deposit.
4. **Geographical Insights:** Portugal leads in bookings and revenue, while distant countries like Madagascar and Nepal show minimal activity.
5. **Seasonality:** January and February see peak bookings, with strong summer seasonality for arrivals in August, July, and May.
6. **Daily Rate Patterns:** Average daily rates peak in July, decline steadily until November, and rebound slightly towards the year-end.
7. **Booking Outcomes:** Most bookings are successfully checked out, though cancellations are common, especially among no-deposit bookings.
8. **Revenue Trends:** Both booking revenue and profits steadily grew from 2013 to 2017, with notable surges in 2014 and 2017.
9. **Arrival Trends:** Fridays and Thursdays see the highest guest arrivals, reflecting a preference for weekend check-ins.
10. **First-Time Visitors:** All guests are first-time visitors, suggesting potential opportunities for enhancing customer loyalty programs.

<left><h1 style="color:darkred; font-size: 36px;">Recommendations</h1></left>



1. **Enhance Loyalty Programs:** Develop initiatives to encourage repeat visits, such as discounts for returning customers, exclusive membership perks, or personalized offers.

2. **Optimize Booking Channels:** Focus on improving profitability from the "Online Travel Agent" channel, such as negotiating better rates or incentives for high-performing partners. Diversify booking sources to reduce dependence on a single channel.

3. **Target Key Markets:** Invest in marketing campaigns in Portugal, the UK, and France to further capitalize on these strong-performing regions. Use customized strategies for low-activity regions to expand your reach.

4. **Plan for Seasonality:** Boost promotions and inventory during peak months like January, February, and summer, while planning strategies to attract guests during low-demand periods.

5. **Refine No-Deposit Policies:** Encourage more refundable or advance deposit bookings to reduce cancellations and ensure commitment.

6. **Upsell Opportunities:** Focus on transient guests and couples with targeted upselling options, such as romantic packages or add-ons for leisure activities.

7. **Revenue Management:** Analyze long-term booking trends to better plan inventory and pricing strategies that optimize profits across lead time categories.

8. **Daily Rate Adjustments:** Maximize revenue during high-demand periods like July by adjusting daily rates. Offer discounts or packages during low-demand periods to maintain steady occupancy.

9. **Expand Weekend Offerings:** Leverage the preference for Friday arrivals by creating special weekend deals or activities that enhance guest experiences.

10. **Boost Digital Presence:** Strengthen digital marketing efforts to maintain dominance in online travel agent bookings, while addressing cancellation issues through improved customer communication or policies.

